In [1]:
from neuroprobe.braintreebank_subject import BrainTreebankSubject
import neuroprobe.train_test_splits as neuroprobe_train_test_splits
import neuroprobe.config as neuroprobe_config

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import torch, numpy as np
import argparse, json, os, time, psutil
import gc  # Add at top with other imports

from eval_utils import *


preprocess_options = [
    'none', # no preprocessing, just raw voltage
    'stft_absangle', # magnitude and phase after FFT
    'stft_realimag', # real and imaginary parts after FFT
    'stft_abs', # just magnitude after FFT ("spectrogram")

    'remove_line_noise', # remove line noise from the raw voltage
    'downsample_200', # downsample to 200 Hz
    'downsample_200-remove_line_noise', # downsample to 200 Hz and remove line noise
]
splits_options = [
    'SS_SM', # same subject, same trial
    'SS_DM', # same subject, different trial
    'DS_DM', # different subject, different trial
]
# Hardcoded parameters based on command line args
eval_name = 'onset'
splits_type = 'SS_DM'
subject_id = 1
trial_id = 1
preprocess_type = 'laplacian-stft_abs'
classifier_type = 'linear'
only_1second = True
verbose = True

# Default values for other parameters
overwrite = False
save_dir = 'eval_results'
seed = 42
lite = True  # Default is lite since --full not specified
nano = False

# Validate nano settings
assert (not nano) or (splits_type != "SS_DM"), "Nano only works with SS_SM or DS_DM splits; does not work with SS_DM."
assert (not nano) or lite, "--nano and --full cannot be used together. Neuroprobe Full and Neuroprobe Nano are different evaluations."

print("starting")

eval_names = eval_name.split(',')

preprocess_parameters = {
    "type": preprocess_type,
    "stft": {
        "nperseg": 512,
        "poverlap": 0.75,
        "window": 'hann',
        "max_frequency": 150
    }
}

classifier_type = "linear"

model_name = model_name_from_classifier_type(classifier_type)

# Set random seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)

bins_start_before_word_onset_seconds = 0.5 if not only_1second else 0
bins_end_after_word_onset_seconds = 1.5 if not only_1second else 1
bin_size_seconds = 0.25
bin_step_size_seconds = 0.125

bin_starts = []
bin_ends = []
if not only_1second:
    for bin_start in np.arange(-bins_start_before_word_onset_seconds, bins_end_after_word_onset_seconds-bin_size_seconds, bin_step_size_seconds):
        bin_end = bin_start + bin_size_seconds
        if bin_end > bins_end_after_word_onset_seconds: break

        bin_starts.append(bin_start)
        bin_ends.append(bin_end)
    bin_starts += [-bins_start_before_word_onset_seconds]
    bin_ends += [bins_end_after_word_onset_seconds]
bin_starts += [0]
bin_ends += [1]


# use cache=True to load this trial's neural data into RAM, if you have enough memory!
# It will make the loading process faster.
subject = BrainTreebankSubject(subject_id, allow_corrupted=False, cache=True, dtype=torch.float32)
if nano:
    all_electrode_labels = neuroprobe_config.NEUROPROBE_NANO_ELECTRODES[subject.subject_identifier]
elif lite:
    all_electrode_labels = neuroprobe_config.NEUROPROBE_LITE_ELECTRODES[subject.subject_identifier]
else:
    all_electrode_labels = subject.electrode_labels
subject.set_electrode_subset(all_electrode_labels)  # Use all electrodes
neural_data_loaded = False

print("1")


starting
1


In [6]:

############## DATA PREPROCESSING ###############

from scipy import signal
import numpy as np

def preprocess_stft(data, sampling_rate=2048, preprocess="stft_abs", preprocess_parameters={"stft": {"nperseg": 512, "poverlap": 0.875}}):
    was_tensor = isinstance(data, torch.Tensor)
    x = torch.from_numpy(data) if not was_tensor else data

    if len(x.shape) == 2: # if it is only (n_electrodes, n_samples)
        x = x.unsqueeze(0)
    # data is of shape (batch_size, n_electrodes, n_samples)
    batch_size, n_electrodes, n_samples = x.shape

    # convert to float32 and reshape for STFT
    x = x.to(dtype=torch.float32)
    x = x.reshape(batch_size * n_electrodes, -1)

    # STFT parameters
    nperseg = preprocess_parameters["stft"]["nperseg"]
    poverlap = preprocess_parameters["stft"]["poverlap"]
    noverlap = int(nperseg * poverlap)
    hop_length = nperseg - noverlap

    if preprocess_parameters["stft"]["window"] == "hann":
        window = torch.hann_window(nperseg, device=x.device)
    elif preprocess_parameters["stft"]["window"] == "boxcar":
        window = torch.ones(nperseg, device=x.device)
    else:
        raise ValueError(f"Invalid window type: {preprocess_parameters['stft']['window']}")
    
    max_frequency = preprocess_parameters["stft"]["max_frequency"]

    print("inputs to stft:", x)

    # Compute STFT
    x = torch.stft(x,
                    n_fft=nperseg, 
                    hop_length=hop_length,
                    win_length=nperseg,
                    window=window,
                    return_complex=True,
                    normalized=False,
                    center=True)

    print("outputs from stft:", x)

    # Get frequency bins
    freqs = torch.fft.rfftfreq(nperseg, d=1.0/sampling_rate) # 2048Hz sampling rate
    x = x[:, freqs <= max_frequency]

    if preprocess == "stft_absangle":
        # Split complex values into magnitude and phase
        magnitude = torch.abs(x)
        phase = torch.angle(x)
        # Stack magnitude and phase along a new axis
        x = torch.stack([magnitude, phase], dim=-2)
    elif preprocess == "stft_realimag":
        real = torch.real(x)
        imag = torch.imag(x)
        x = torch.stack([real, imag], dim=-2)
    elif preprocess == "stft_abs":   
        x = torch.abs(x)
    else:
        raise ValueError(f"Invalid preprocess type: {preprocess}")

    print("after mfb and abs:", x)

    # Reshape back
    _, n_freqs, n_times = x.shape
    x = x.reshape(batch_size, n_electrodes, n_freqs, n_times)
    x = x.transpose(2, 3) # (batch_size, n_electrodes, n_timebins, n_freqs)
    
    # Z-score normalization
    x = x - x.mean(dim=[0, 2], keepdim=True)
    x = x / (x.std(dim=[0, 2], keepdim=True) + 1e-5)

    print("after z-score normalization:", x)

    return x.numpy() if not was_tensor else x

def downsample(data, fs=2048, downsample_rate=200):
    # Handle both numpy arrays and torch tensors
    was_tensor = isinstance(data, torch.Tensor)
    if was_tensor:
        device = data.device
        data_np = data.cpu().numpy()
    else:
        data_np = data
    
    # Apply downsampling
    result = signal.resample_poly(data_np, up=fs, down=downsample_rate, axis=-1)
    
    # Convert back to tensor if input was a tensor
    if was_tensor:
        result = torch.from_numpy(result).to(device)
    
    return result
def remove_line_noise(data, fs=2048, line_freq=60):
    """Remove line noise (60 Hz and harmonics) from neural data.
    
    Args:
        data (numpy.ndarray or torch.Tensor): Input voltage data of shape (n_channels, n_samples) or (batch_size, n_channels, n_samples)
        fs (int): Sampling frequency in Hz
        line_freq (int): Fundamental line frequency in Hz (typically 60 Hz in the US)
        
    Returns:
        numpy.ndarray or torch.Tensor: Filtered data with the same shape as input (same type as input)
    """
    # Handle both numpy arrays and torch tensors
    was_tensor = isinstance(data, torch.Tensor)
    if was_tensor:
        device = data.device
        filtered_data = data.cpu().numpy().copy()
    else:
        filtered_data = data.copy()
    
    # Define the width of the notch filter (5 Hz on each side)
    bandwidth = 5.0
    
    # Calculate the quality factor Q
    Q = line_freq / bandwidth
    
    # Apply notch filters for the fundamental frequency and harmonics
    # We'll filter up to the 5th harmonic (60, 120, 180, 240, 300 Hz)
    for harmonic in range(1, 6):
        harmonic_freq = line_freq * harmonic
        
        # Skip if the harmonic frequency is above the Nyquist frequency
        if harmonic_freq > fs/2:
            break
            
        # Create and apply a notch filter
        b, a = signal.iirnotch(harmonic_freq, Q, fs)
        
        # Apply the filter along the time dimension
        if filtered_data.ndim == 2:  # (n_channels, n_samples)
            filtered_data = signal.filtfilt(b, a, filtered_data, axis=1)
        elif filtered_data.ndim == 3:  # (batch_size, n_channels, n_samples)
            for i in range(filtered_data.shape[0]):
                filtered_data[i] = signal.filtfilt(b, a, filtered_data[i], axis=1)
    
    # Convert back to tensor if input was a tensor
    if was_tensor:
        filtered_data = torch.from_numpy(filtered_data).to(device)
    
    return filtered_data

def laplacian_rereference_neural_data(electrode_data, electrode_labels, remove_non_laplacian=True):
    """
    Rereference the neural data using the laplacian method (subtract the mean of the neighbors, as determined by the electrode labels)
    inputs:
        electrode_data: torch tensor of shape (batch_size, n_electrodes, n_samples) or (n_electrodes, n_samples)
        electrode_labels: list of electrode labels
        remove_non_laplacian: boolean, if True, remove the non-laplacian electrodes from the data; if false, keep them without rereferencing
    outputs:
        rereferenced_data: torch tensor of shape (batch_size, n_electrodes_rereferenced, n_samples) or (n_electrodes_rereferenced, n_samples)
        rereferenced_labels: list of electrode labels of length n_electrodes_rereferenced (n_electrodes_rereferenced could be different from n_electrodes if remove_non_laplacian is True)
    """
    def get_all_laplacian_electrodes(electrode_labels):
        """
            Get all laplacian electrodes for a given subject. This function is originally from
            https://github.com/czlwang/BrainBERT repository (Wang et al., 2023)
        """
        def stem_electrode_name(name):
            #names look like 'O1aIb4', 'O1aIb5', 'O1aIb6', 'O1aIb7'
            #names look like 'T1b2
            found_stem_end = False
            stem, num = [], []
            for c in reversed(name):
                if c.isalpha():
                    found_stem_end = True
                if found_stem_end:
                    stem.append(c)
                else:
                    num.append(c)
            return ''.join(reversed(stem)), int(''.join(reversed(num)))
        def has_neighbors(stem, stems):
            (x,y) = stem
            return ((x,y+1) in stems) or ((x,y-1) in stems)
        def get_neighbors(stem, stems):
            (x,y) = stem
            return [f'{x}{y}' for (x,y) in [(x,y+1), (x,y-1)] if (x, y) in stems]
        stems = [stem_electrode_name(e) for e in electrode_labels]
        laplacian_stems = [x for x in stems if has_neighbors(x, stems)]
        electrodes = [f'{x}{y}' for (x,y) in laplacian_stems]
        neighbors = {e: get_neighbors(stem_electrode_name(e), stems) for e in electrodes}
        return electrodes, neighbors

    # Handle both numpy arrays and torch tensors
    was_tensor = isinstance(electrode_data, torch.Tensor)

    batch_unsqueeze = False
    if len(electrode_data.shape) == 2:
        batch_unsqueeze = True
        if was_tensor:
            electrode_data = electrode_data.unsqueeze(0)
        else:
            electrode_data = electrode_data[np.newaxis, :, :]

    laplacian_electrodes, laplacian_neighbors = get_all_laplacian_electrodes(electrode_labels)
    laplacian_neighbor_indices = {laplacian_electrode_label: [electrode_labels.index(neighbor_label) for neighbor_label in neighbors] for laplacian_electrode_label, neighbors in laplacian_neighbors.items()}

    batch_size, n_electrodes, n_samples = electrode_data.shape
    rereferenced_n_electrodes = len(laplacian_electrodes) if remove_non_laplacian else n_electrodes
    if was_tensor:
        rereferenced_data = torch.zeros((batch_size, rereferenced_n_electrodes, n_samples), dtype=electrode_data.dtype, device=electrode_data.device)
    else:
        rereferenced_data = np.zeros((batch_size, rereferenced_n_electrodes, n_samples), dtype=electrode_data.dtype)

    electrode_i = 0
    original_electrode_indices = []
    for original_electrode_index, electrode_label in enumerate(electrode_labels):
        if electrode_label in laplacian_electrodes:
            rereferenced_data[:, electrode_i] = electrode_data[:, electrode_i] - electrode_data[:, laplacian_neighbor_indices[electrode_label]].mean(axis=1)
            original_electrode_indices.append(original_electrode_index)
            electrode_i += 1
        else:
            if remove_non_laplacian: 
                continue # just skip the non-laplacian electrodes
            else:
                rereferenced_data[:, electrode_i] = electrode_data[:, electrode_i]
                original_electrode_indices.append(original_electrode_index)
                electrode_i += 1
                
    if batch_unsqueeze:
        if was_tensor:
            rereferenced_data = rereferenced_data.squeeze(0)
        else:
            rereferenced_data = rereferenced_data.squeeze(0)

    return rereferenced_data, laplacian_electrodes if remove_non_laplacian else electrode_labels, original_electrode_indices

def preprocess_data(data, electrode_labels, preprocess, preprocess_parameters):
    for preprocess_option in preprocess.split('-'):
        print(f"data right before preprocess_option {preprocess_option}:", data)
        if preprocess_option.lower() in ['stft_absangle', 'stft_realimag', 'stft_abs']:
            data = preprocess_stft(data, preprocess=preprocess_option, preprocess_parameters=preprocess_parameters)
        elif preprocess_option.lower() == 'remove_line_noise':
            data = remove_line_noise(data)
        elif preprocess_option.lower() == 'downsample_200':
            data = downsample(data, downsample_rate=200)
        elif preprocess_option.lower() == 'laplacian':
            data, electrode_labels, original_electrode_indices = laplacian_rereference_neural_data(data, electrode_labels, remove_non_laplacian=False)
    return data

In [12]:

for eval_name in eval_names:
    start_time = time.time()

    preprocess_suffix = f"{preprocess_type}" if preprocess_type != 'none' else 'voltage'
    preprocess_suffix += f"_nperseg{preprocess_parameters['stft']['nperseg']}" if preprocess_type.startswith('stft') else ''
    preprocess_suffix += f"_poverlap{preprocess_parameters['stft']['poverlap']}" if preprocess_type.startswith('stft') else ''
    preprocess_suffix += f"_{preprocess_parameters['stft']['window']}" if preprocess_type.startswith('stft') and preprocess_parameters['stft']['window'] != 'hann' else ''
    preprocess_suffix += f"_maxfreq{preprocess_parameters['stft']['max_frequency']}" if preprocess_type.startswith('stft') and preprocess_parameters['stft']['max_frequency'] != 200 else ''

    file_save_dir = f"{save_dir}/{classifier_type}_{preprocess_suffix}"
    os.makedirs(file_save_dir, exist_ok=True) # Create save directory if it doesn't exist

    file_save_path = f"{file_save_dir}/population_{subject.subject_identifier}_{trial_id}_{eval_name}.json"
    if os.path.exists(file_save_path) and not overwrite:
        log(f"Skipping {file_save_path} because it already exists", priority=0)
        continue

    # Load neural data if it hasn't been loaded yet; NOTE: this is done here to avoid unnecessary loading of neural data if the file is going to be skipped.
    if not neural_data_loaded:
        start_time = time.time()
        subject.load_neural_data(trial_id)
        subject_load_time = time.time() - start_time
        if verbose:
            log(f"Subject loaded in {subject_load_time:.2f} seconds", priority=0)
        neural_data_loaded = True

    results_population = {
        "time_bins": [],
    }

    # train_datasets and test_datasets are arrays of length k_folds, each element is a BrainTreebankSubjectTrialBenchmarkDataset for the train/test split
    if splits_type == "SS_SM":
        train_datasets, test_datasets = neuroprobe_train_test_splits.generate_splits_SS_SM(subject, trial_id, eval_name, dtype=torch.float32, 
                                                                                        output_indices=False, 
                                                                                        start_neural_data_before_word_onset=int(bins_start_before_word_onset_seconds*neuroprobe_config.SAMPLING_RATE), 
                                                                                        end_neural_data_after_word_onset=int(bins_end_after_word_onset_seconds*neuroprobe_config.SAMPLING_RATE),
                                                                                        lite=lite, nano=nano, allow_partial_cache=True)
    elif splits_type == "SS_DM":
        train_datasets, test_datasets = neuroprobe_train_test_splits.generate_splits_SS_DM(subject, trial_id, eval_name, dtype=torch.float32, 
                                                                                        output_indices=False, 
                                                                                        start_neural_data_before_word_onset=int(bins_start_before_word_onset_seconds*neuroprobe_config.SAMPLING_RATE), 
                                                                                        end_neural_data_after_word_onset=int(bins_end_after_word_onset_seconds*neuroprobe_config.SAMPLING_RATE),
                                                                                        lite=lite, allow_partial_cache=True)
        train_datasets = [train_datasets]
        test_datasets = [test_datasets]
    elif splits_type == "DS_DM":
        if verbose: log("Loading the training subject...", priority=0)
        train_subject_id = neuroprobe_config.DS_DM_TRAIN_SUBJECT_ID
        train_subject = BrainTreebankSubject(train_subject_id, allow_corrupted=False, cache=True, dtype=torch.float32)
        train_subject_electrodes = neuroprobe_config.NEUROPROBE_LITE_ELECTRODES[train_subject.subject_identifier] if lite else train_subject.electrode_labels
        train_subject.set_electrode_subset(train_subject_electrodes)
        all_subjects = {
            subject_id: subject,
            train_subject_id: train_subject,
        }
        if verbose: log("Subject loaded.", priority=0)
        train_datasets, test_datasets = neuroprobe_train_test_splits.generate_splits_DS_DM(all_subjects, subject_id, trial_id, eval_name, dtype=torch.float32, 
                                                                                        output_indices=False, 
                                                                                        start_neural_data_before_word_onset=int(bins_start_before_word_onset_seconds*neuroprobe_config.SAMPLING_RATE), 
                                                                                        end_neural_data_after_word_onset=int(bins_end_after_word_onset_seconds*neuroprobe_config.SAMPLING_RATE),
                                                                                        lite=lite, nano=nano, allow_partial_cache=True)
        train_datasets = [train_datasets]
        test_datasets = [test_datasets]


    for bin_start, bin_end in zip(bin_starts, bin_ends):
        data_idx_from = int((bin_start+bins_start_before_word_onset_seconds)*neuroprobe_config.SAMPLING_RATE)
        data_idx_to = int((bin_end+bins_start_before_word_onset_seconds)*neuroprobe_config.SAMPLING_RATE)

        bin_results = {
            "time_bin_start": float(bin_start),
            "time_bin_end": float(bin_end),
            "folds": []
        }

        # Loop over all folds
        for fold_idx in range(len(train_datasets)):
            train_dataset = train_datasets[fold_idx]
            test_dataset = test_datasets[fold_idx]

            log(f"Fold {fold_idx+1}, Bin {bin_start}-{bin_end}")
            log("Preparing and preprocessing data...", priority=2, indent=1)
            batch_size = 1000
            batch_items = torch.concatenate([train_dataset[i][0][:, data_idx_from:data_idx_to].unsqueeze(0) for i in range(min(batch_size, len(train_dataset)))]).float()

            print(f"First {batch_size} items in train_dataset:", batch_items)
            preprocessed_batch = preprocess_data(batch_items, all_electrode_labels, preprocess_type, preprocess_parameters).float().numpy()
            print(f"Preprocessed batch from train_dataset (shape: {preprocessed_batch.shape}):", preprocessed_batch)


            # Convert PyTorch dataset to numpy arrays for scikit-learn
            X_train = np.concatenate([preprocess_data(item[0][:, data_idx_from:data_idx_to].unsqueeze(0), all_electrode_labels, preprocess_type, preprocess_parameters).float().numpy() for item in train_dataset], axis=0)
            y_train = np.array([item[1] for item in train_dataset])
            X_test = np.concatenate([preprocess_data(item[0][:, data_idx_from:data_idx_to].unsqueeze(0), all_electrode_labels, preprocess_type, preprocess_parameters).float().numpy() for item in test_dataset], axis=0)
            y_test = np.array([item[1] for item in test_dataset])
            gc.collect()  # Collect after creating large arrays

            print("X_train.shape after preprocessing:", X_train.shape)
            print(X_train)

            if splits_type == "DS_DM":
                if verbose: log("Combining regions...", priority=2, indent=1)
                regions_train = get_region_labels(train_subject)
                regions_test = get_region_labels(subject)
                X_train, X_test, common_regions = combine_regions(X_train, X_test, regions_train, regions_test)

            print("X_train.shape just before flattening:", X_train.shape)
            print("X_train")

            # Flatten the data after preprocessing in-place
            original_X_train_shape = X_train.shape
            original_X_test_shape = X_test.shape
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)

            log(f"Standardizing data...", priority=2, indent=1)

            # Standardize the data in-place
            scaler = StandardScaler(copy=False)
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            gc.collect()  # Collect after standardization

            log(f"Training model...", priority=2, indent=1)

            print("X_train.shape just before training:", X_train.shape)
            print("X_train")

            break
        break
    break

[14:22:58 gpu 00.0G ram 021.2G] Fold 1, Bin 0-1
[14:22:58 gpu 00.0G ram 021.2G]     Preparing and preprocessing data...
First 1000 items in train_dataset: tensor([[[ 14.8872,  15.4189,   9.3045,  ...,  99.1593,  95.4375,  94.6400],
         [-32.1670, -31.9011, -35.0912,  ..., 105.2737, 102.8811, 102.0836],
         [ 36.4204,  34.5595,  32.4328,  ...,  52.6369,  49.7126,  47.3200],
         ...,
         [-54.7636, -52.9027, -55.5611,  ...,  -7.9753, -10.1020, -12.7604],
         [-35.8888, -36.6863, -35.6229,  ..., -24.4575, -26.3184, -25.7867],
         [ 17.5456,  15.6847,  13.8238,  ...,  53.7002,  48.3834,  44.1299]],

        [[-20.4699, -25.5209, -21.0016,  ..., -50.7760, -59.0171, -67.2582],
         [ 48.6492,  44.3957,  51.3076,  ...,   0.2658,  -4.7852, -14.0897],
         [-71.5117, -76.2969, -67.2582,  ..., -39.3447, -40.6739, -45.7249],
         ...,
         [ 38.5472,  36.9521,  39.3447,  ...,  -0.0000,  -4.2535,  -8.5070],
         [ 40.6739,  42.5348,  42.0031,  ...,

KeyboardInterrupt: 

In [4]:
data_idx_from, data_idx_to

(1024, 3072)

In [3]:
all_electrode_labels

['F3aOFa7',
 'F3aOFa8',
 'F3aOFa9',
 'F3aOFa10',
 'F3aOFa11',
 'F3aOFa12',
 'F3aOFa13',
 'F3aOFa14',
 'F3aOFa15',
 'F3aOFa16',
 'F3aOFa2',
 'F3aOFa3',
 'F3aOFa4',
 'T1bIc1',
 'T1bIc2',
 'T1bIc3',
 'T1bIc4',
 'T1bIc5',
 'T1bIc6',
 'T1bIc7',
 'T1bIc8',
 'F3bIaOFb1',
 'F3bIaOFb2',
 'F3bIaOFb3',
 'F3bIaOFb4',
 'F3bIaOFb5',
 'F3bIaOFb6',
 'F3bIaOFb7',
 'F3bIaOFb8',
 'F3bIaOFb9',
 'F3bIaOFb10',
 'F3bIaOFb11',
 'F3bIaOFb12',
 'F3bIaOFb13',
 'F3bIaOFb14',
 'F3bIaOFb15',
 'F3bIaOFb16',
 'T1aIb1',
 'T1aIb2',
 'T1aIb3',
 'T1aIb4',
 'T1aIb5',
 'T1aIb6',
 'T1aIb7',
 'T1aIb8',
 'T2aA1',
 'T2aA2',
 'T2aA3',
 'T2aA4',
 'T2aA5',
 'T2aA6',
 'T2aA7',
 'T2aA8',
 'T2aA9',
 'T2aA10',
 'T2aA11',
 'T2aA12',
 'T3aHb9',
 'T3aHb10',
 'T3aHb6',
 'T3aHb12',
 'T1cIf1',
 'T1cIf2',
 'T1cIf3',
 'T1cIf4',
 'T1cIf5',
 'T1cIf6',
 'T1cIf7',
 'T1cIf8',
 'T1cIf10',
 'T1cIf11',
 'T1cIf12',
 'T1cIf13',
 'T1cIf14',
 'T1cIf15',
 'T1cIf16',
 'T2bHa7',
 'T2bHa8',
 'T2bHa9',
 'T2bHa10',
 'T2bHa11',
 'T2bHa12',
 'T2bHa13',
 'T2bHa1